In [1]:

import numpy as np
import matplotlib.pyplot as plt
import sys
import os
from tqdm import tqdm
sys.path.append('..')

In [2]:

import numpy as np
import matplotlib.pyplot as plt
import sys
import os
from tqdm import tqdm
sys.path.append('..')

In [3]:
%load_ext autoreload
%autoreload 2
from solat_cb.simulation import *
from solat_cb.spectra import *
from solat_cb.mle import *

In [4]:
libdir ='/pscratch/sd/l/lonappan/SOLAT'
nside = 1024
cb_method = 'iso'
beta = 0.35
dust = 10
synch = 5
alpha = [-0.1,-0.1,0.2,0.2,.15,.15]
atm_noise = True
nsplits = 2
bandpass = False
fit = "As + Asd + Ad + beta + alpha"
binwidth = 20
bmin = 60
bmax = 2000

In [5]:
lat = LATsky(libdir,nside,cb_method,dust,synch,alpha,beta,atm_noise=atm_noise,nsplits=nsplits,bandpass=bandpass)

CMB : INFO - Loading CMB power spectra from file
Noise : INFO - Noise Model: White + 1/f noise v3.0.0


In [10]:
spec = Spectra(lat,cache=True,parallel=0,)

Spectra : INFO - Reading apodised mask from /pscratch/sd/l/lonappan/SOLAT/LAT_atm_noise_2splits/spectra_1024_aposcale2p0_common/workspaces/mask_N1024_aposcale2p0_CO_PS.fits
Spectra : INFO - Reading coupling Matrix from /pscratch/sd/l/lonappan/SOLAT/LAT_atm_noise_2splits/spectra_1024_aposcale2p0_common/workspaces/coupling_matrix_N1024_fsky0p55_aposcale2p0_CO_PS.fits
Spectra : INFO - No parallelization


In [11]:
ml = MLE(libdir,spec,fit, alpha_per_split=False,rm_same_tube=False,binwidth=binwidth,bmin=bmin,bmax=bmax)

CMB : INFO - Loading CMB power spectra from file


Fitting a common polarisation angle per frequency


In [12]:
ml.estimate_angles(0)

{'As': 2.2392708431683235,
 'Asd': 1.0133189320428817,
 'Ad': 0.9226392457953934,
 'beta': 0.28458740971045743,
 '27': 0.04590011833803151,
 '39': 0.057542118812855335,
 '93': 0.25795747987490747,
 '145': 0.2592744108309074,
 '225': 0.17710366990047585,
 '280': 0.17709276676647362}

In [13]:
LowStat(ml).estimated_params()

{'beta': {'mean': 0.3609884868961907, 'std': 0.10207734969780047},
 '27': {'mean': -0.11251814419881984, 'std': 0.11613327703243445},
 '39': {'mean': -0.11460844527538133, 'std': 0.1120332546280248},
 '93': {'mean': 0.18820683500647092, 'std': 0.10207751953105941},
 '145': {'mean': 0.18748962091701388, 'std': 0.09979563912984826},
 '225': {'mean': 0.13759716163350752, 'std': 0.1004659564153729},
 '280': {'mean': 0.14219410790948214, 'std': 0.1023421574865071}}